In [3]:
import transformers
import torch 
import gc 
import json 
from tokens import * 

In [5]:
gc.collect()
torch.cuda.empty_cache()

In [2]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]
/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-11-07 17:54:01.205163: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical

In [8]:
def repeater(messages):

    # Apply chat template to messages
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Define terminators
    terminators = [
        pipeline.tokenizer.eos_token_id,
    ]

    # Generate text
    outputs = pipeline(
        prompt,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        max_new_tokens=2024*2
    )
    return len(prompt), outputs

In [9]:
with open('articles.json', 'r') as file: 
    articles = json.load(file)

print(f'{len(articles[len(articles) - 1])} are available.')

5 are available.


In [10]:
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing NEUTRAL news article based on the provided article information."
                                "Try to filter the bias that maybe existent in the article" 
                                "Write a new article as a journalist, ensuring the content is unbiased, factual, and informative."},
                                
    {"role": "user", "content": articles[0][0]}
]

In [11]:
prompt_len, outputs = repeater(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [12]:
print(outputs[0]["generated_text"][prompt_len:], end='.')


 Title: Starmer Updates Ministerial Gift and Hospitality Rules in Response to Public Scrutiny

Keir Starmer, the Prime Minister, has tightened the rules regarding gifts and hospitality accepted by ministers, following a series of criticisms over the acceptance of freebies by senior Labour figures. The updated ministerial code now requires ministers to consider the "need to maintain the public's confidence in the standards of propriety" when deciding whether to accept gifts.

The new regulations state that ministers should refrain from accepting gifts, hospitality, or services that may compromise their judgment or create an appearance of obligation. This principle extends to family members as well. However, the code acknowledges that ministers may need to attend events where hospitality is offered due to their governmental duties.

The monthly register of gifts received by ministers, which will now be published, will include details and the value of gifts worth more than £140, as well a

In [13]:
del outputs
gc.collect()
torch.cuda.empty_cache()

In [21]:
del model